In [2]:
import sqlite3

In [3]:
con = sqlite3.connect("via_data_test.db")
cur = con.cursor()

# Create tbl_location

In [3]:
cur.execute('''
          CREATE TABLE tbl_location_v2 (
            LOCATION_ID INTEGER PRIMARY KEY,
            LOCATION text UNIQUE NOT NULL,
            LOCATION_CODE text UNIQUE NOT NULL,
            DATE_CREATED datetime default current_timestamp)
          ''')

# Create tbl_route

In [7]:
cur.execute('''
          CREATE TABLE tbl_route_v3 (
            ROUTE_ID INTEGER PRIMARY KEY,
            ROUTE_START INTEGER NOT NULL REFERENCES tbl_location(LOCATION_ID),
            ROUTE_END INTEGER NOT NULL REFERENCES tbl_location(LOCATION_ID),
            DATE_CREATED datetime default current_timestamp,
            CONSTRAINT unique_route UNIQUE (ROUTE_START, ROUTE_END))
          ''')

In [5]:
cur.execute('''
          DROP TABLE tbl_route;
          ''')

# Create tbl_route

In [6]:
cur.execute('''
          CREATE TABLE tbl_train_v4 (
            TRAIN_ID INTEGER PRIMARY KEY,
            TRAIN_NUMBER text NOT NULL,
            ROUTE INTEGER NOT NULL REFERENCES tbl_route(ROUTE_ID),
            DATE_CREATED datetime default current_timestamp,
            CONSTRAINT unique_route UNIQUE (TRAIN_NUMBER, ROUTE))
          ''')

# create tbl_stop

In [5]:
cur.execute('''
          CREATE TABLE tbl_stop_v5 (
            STOP_ID INTEGER PRIMARY KEY,
            TRAIN INTEGER NOT NULL REFERENCES tbl_train(TRAIN_ID),
            ROUTE_STOP INTEGER NOT NULL REFERENCES tbl_location(LOCATION_ID),
            ROUTE_PREV_STOP INTEGER NOT NULL REFERENCES tbl_location(LOCATION_ID),
            DATE_CREATED datetime default current_timestamp,
            CONSTRAINT unique_route UNIQUE (TRAIN, ROUTE_STOP, ROUTE_PREV_STOP))
          ''')

# create tbl_via_data

In [6]:
cur.execute('''
          CREATE TABLE tbl_via_data_v4 (
            DATA_ID INTEGER PRIMARY KEY,
            TRAIN_STOP INTEGER NOT NULL REFERENCES tbl_stop(STOP_ID),
            SCHEDULE_DATETIME DATETIME NOT NULL,
            ARRIVAL_DATETIME DATETIME NOT NULL,
            MINUTES_LATE INTEGER NOT NULL,
            DATE_TRAIN DATETIME NOT NULL,
            DATE_CREATED DATETIME default current_timestamp,
            CONSTRAINT unique_route UNIQUE (TRAIN_STOP, DATE_TRAIN))
          ''')

# Create trigger
https://stackoverflow.com/questions/19337029/insert-if-not-exists-statement-in-sqlite

In [ ]:
cur.execute('''
        CREATE TRIGGER via_data_before_insert
        BEFORE INSERT ON tbl_via_data
        FOR EACH ROW
        BEGIN
            INSERT OR IGNORE INTO bookmarks(LOCATION) VALUES(new.location);
        END;
          ''')

In [ ]:
# Create data trigger

cur.execute('''
          CREATE TRIGGER via_data_before_insert
          BEFORE INSERT ON tbl_via_data
          FOR EACH ROW
          BEGIN
            if new.location is not null then
              LCL_location := new.location;
              SELECT LOCATION_ID INTO LCL_location_id FROM tbl_location WHERE LOCATION = LCL_location;
              WHEN NO_DATA_FOUND THEN
                INSERT INTO tbl_location (LOCATION) VALUES (LCL_location);
                RETURNING LOCATION_ID INTO LCL_location_id;
              END;
            new.location_id := LCL_location_id;
            end if;
          END;
          ''')

# create data view
## used for downstream analysis / dashboard

In [16]:
cur.execute('''
          CREATE VIEW view_via_data 
            AS 
            SELECT * FROM tbl_via_data
          ''')